In [8]:
import polars as pl

df = pl.DataFrame(
    {
        "foo": [1, 2, 3, 4, 5, 6],
        "ham": ["a", "b", "c", None, "e", "f"],
    }
)

In [10]:
(
    df
    .select(
        (
            pl.col('ham')
            .cast(pl.Categorical)
            .to_physical()
            .cast(pl.Int32)
            .add(10)
            .fill_null(0)
            .alias('ham')
        )
    )
)

ham
i32
10
11
12
0
13
14
